In [1]:
import os
import openai
import sys
sys.path.append('../..')

import constants
openai.api_key = constants.OPENAI_API_KEY

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'

In [4]:
embedding = OpenAIEmbeddings(openai_api_key=constants.OPENAI_API_KEY)
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [6]:
print(vectordb._collection.count())

104


In [7]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [8]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The document the is from, should be one of ./DB004.pdf",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the database",
        type="integer",
    ),
]

In [ ]:
document_content_description = "Database notes"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)